In [1]:
import numpy as np 
from implementations import *
from helpers import *
from helper_functions import *
from optimization import *
%load_ext autoreload
%autoreload 2

In [2]:
# load the training dataset
print("loading the training dataset")
DATA_TRAIN_PATH = 'train.csv'
y_tr, tx_tr, ids_tr = load_csv_data(DATA_TRAIN_PATH)
print("training dataset loaded")

loading the training dataset
training dataset loaded


In [3]:
print(tx_tr.shape)

(250000, 30)


In [13]:
# load the test dataset
print("loading the test dataset")
DATA_TEST_PATH = 'test.csv'
y_te, tx_te, ids_te = load_csv_data(DATA_TEST_PATH)
print("test dataset loaded")

loading the test dataset
test dataset loaded


# With splitting

## Data cleaning

In [4]:
txOpti, yOpti, idsOpti = dataClean(tx_tr, y_tr, r=4)

## Train using ridge regression

In [5]:
degree = 4

losses = []
ws = []
txPoly = [] 

for i in range(4): 
    yOpti[i][np.where(yOpti[i] == -1)] = 0

    t = build_poly(txOpti[i], degree)
    w, loss = ridge_regression(yOpti[i], t, 0.000001)
    losses.append(loss) 
    ws.append(w) 
    txPoly.append(t)
    print(f'end of the ridge_regression with loss={loss}')
    
    predicted = predict(w, t)
    yOpti[i][np.where(yOpti[i] == 0)] = -1
    acc = calculate_accuracy(yOpti[i], predicted)
    print(f'{acc:.15f}')

end of the ridge_regression with loss=0.059591625909013804
0.837458588972406
end of the ridge_regression with loss=0.07592092592146824
0.794452181987001
end of the ridge_regression with loss=0.06801183472127788
0.821235038408861
end of the ridge_regression with loss=0.07174975404874817
0.812804547915539


## Test accuracy (train)

In [6]:
y_pred = np.zeros(len(ids_tr))
degree = 4

for i in range(4):
    predicted = predict(ws[i], txPoly[i])
    y_pred[idsOpti[i]] = predicted

y_pred[np.where(y_pred == 0)] = -1

acc = calculate_accuracy(y_tr, y_pred)
print(f'{acc:.15f}')

0.818664000000000


## Predict with test set

In [31]:
txOpti_te, yOpti_te, idsOpti_te = dataClean(tx_te, y_te)

In [33]:
y_pred_te = np.zeros(len(ids_te))
degree = 4

for i in range(4):
    t = build_poly(txOpti_te[i], degree)
    predicted = predict(ws[i], t)
    y_pred_te[idsOpti_te[i]] = predicted

# Without split

## Data cleaning

In [8]:
txOpti_ns = dataClean_without_splitting(tx_tr, r=4)
y_tr_ns = y_tr
y_tr_ns[np.where(y_tr_ns == -1)] = 0

## Train using Ridge regression

In [9]:
# test ridge regression 

t_ns = build_poly(txOpti_ns, 4)

w_ns, loss = ridge_regression(y_tr_ns, t_ns, 0.000001)

print("end of the logistic_regression with loss=", loss)

end of the logistic_regression with loss= 0.07112917164481461


In [10]:
# test ridge regression 

pred_tr_ns = predict(w_ns, t_ns)
y_tr_ns[np.where(y_tr_ns == 0)] = -1
acc = calculate_accuracy(y_tr_ns, pred_tr_ns)

print("the accuracy on the train set is ", acc)

the accuracy on the train set is  0.807472


In [44]:
txOpti_te_ns = dataClean_without_splitting(tx_te)
y_pred_te_ns = predict(w_ns, build_poly(txOpti_te_ns, 4))

In [45]:
acc_12_te = calculate_accuracy(y_pred_te, y_pred_te_ns)
print(acc_12_te)

0.7772412263875348


In [46]:
for i in range(4):
    acc_12 = calculate_accuracy(y_pred[idsOpti[i]], pred_tr_ns[idsOpti[i]])
    print(acc_12)
acc_12 = calculate_accuracy(y_pred, pred_tr_ns)
print(f'compare both full predictions: {acc_12}')

0.8385995816360233
0.7567961415454452
0.7238730423390699
0.8322956145100162
compare both full predictions: 0.789548


In [29]:
for i in range(4):
    oui = [x==y for x, y in zip(y_pred[idsOpti_te[i]],label_te[idsOpti_te[i]])]
    print(f'{sum(oui)*100/len(idsOpti_te[i][0])}')
    print(calculate_accuracy(y_pred[idsOpti_te[i]], label_te[idsOpti_te[i]]))


90.94558116223655
0.9094558116223654
92.52073138737752
0.9252073138737752
86.79348963784803
0.8679348963784802
80.18466748041108
0.8018466748041108


In [39]:
oui1 = 0
oui2 = 0
for i in range(4):
    o1 = [x==1 for x in y_pred[idsOpti_te[i]]]
    o2 = [x==1 for x in label_te[idsOpti_te[i]]]
    oui1 += sum(o1)
    oui2 += sum(o2)
print(f'{oui1} : {oui2}')
    

161349 : 163302


# Test accuracy

In [8]:
labels = []
accs = []
for i in range(4): 
    label = predict(ws[i], txPoly[i])
    yOpti[i][np.where(yOpti[i] == 0)] = -1
    acc = calculate_accuracy(yOpti[i], label)
    print("the accuracy on the train set is ", acc)
    accs.append(acc)
    labels.append(label)

accTot = (accs[0] + accs[1] + accs[2] + accs[3])/4
print("the total accuracy on the train set is ", accTot)

the accuracy on the train set is  0.8202035771120875
the accuracy on the train set is  0.7517280511709481
the accuracy on the train set is  0.7763353778360031
the accuracy on the train set is  0.787222523010287
the total accuracy on the train set is  0.7838723822823314


# Predict test set

In [10]:
txOpti_te, yOpti_te, idsOpti_te = dataClean(tx_te, y_te)

In [11]:
print(tx_te.shape)
print(tx_tr.shape)
print(y_te.shape)
print(y_tr.shape)

(568238, 30)
(250000, 30)
(568238,)
(250000,)


In [29]:
test0 = tx_te[idsOpti_te[0]]
test1 = tx_te[idsOpti_te[1]]
test2 = tx_te[idsOpti_te[2]]
test3 = tx_te[idsOpti_te[3]]
print(np.mean(test0, axis=0)[22])
print(np.mean(test1, axis=0)[22])
print(np.mean(test2, axis=0)[22])
print(np.mean(test3, axis=0)[22])

0.0
1.0
2.0
3.0


In [33]:
a = np.arange(10)
b = [1, 3, 5]
c = [10, 11, 12]
a[b] = c
print(a)

[ 0 10  2 11  4 12  6  7  8  9]


In [19]:
y_pred = np.zeros(len(ids_tr))
degree = 2

for i in range(4):
    print(i)
    t = build_poly(txOpti[i], degree)
    predicted = predict(ws[i], t)
    # predicted[np.where(predicted == 0)] = -1
    y_pred[idsOpti[i]] = predicted

acc = calculate_accuracy(y_tr[0:2], y_pred[0:2])
print(f'{acc:.15f}')

0
1
2
3
0.500000000000000


In [15]:
xo = dataClean_without_splitting(tx_te)
label_te = predict(w, build_poly(xo, 2))

In [42]:
print(yOpti

161349


In [19]:
print(len(ids_te))
print(max(ids_te))

568238
918237


# Generated submission file

In [47]:
OUTPUT_PATH = 'ridge-submission'
create_csv_submission(ids_te, y_pred_te, OUTPUT_PATH)

0.8329537975285004
